# import necessary stuff

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

import spacy
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
import re
from bs4 import BeautifulSoup
import unicodedata
import en_core_web_lg
import en_core_web_sm
from contractions import contractions_dict

import twint
import csv
import sys
from datetime import datetime
import logging

from textstat.textstat import textstatistics, easy_word_set, legacy_round
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
import statsmodels
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

#python -m spacy download en
#python -m spacy download en_core_web_sm
#python -m spacy download en_core_web_lg

## Packages used by script to extract tweets for the URLs: scriptExtractTweets1.py
# import pandas as pd
# import twint
# import csv
# #import os
# import sys
# from datetime import datetime
# import logging

## Packages used by script to create Readability features: scriptCreateReadabilityFeatures1.py
# import pandas as pd
# import numpy as np
# import spacy
# import nltk
# from nltk.tokenize.toktok import ToktokTokenizer
# import re
# from bs4 import BeautifulSoup
# import unicodedata
# import en_core_web_lg
# import en_core_web_sm
# from contractions import contractions_dict
# from textstat.textstat import textstatistics, easy_word_set, legacy_round
# from nltk.corpus import stopwords
# from nltk.tokenize import sent_tokenize, word_tokenize
# from sklearn.preprocessing import MinMaxScaler
# import csv
# import sys
# from datetime import datetime
# import logging

In [2]:
pd.set_option('max_columns', 500)
pd.set_option('max_colwidth', -1)
pd.set_option('max_rows', 500)

# Input file with the URLs to classify
## domain, url, urlType, title, content

In [3]:
urlsFile4Demo = '/home/rohit/SRH/CaseStudy1/dataDemo/dataFiles/demorun_input_data.csv'

# using twint extract the tweets that have the url

In [4]:
# Use this script to process the datasets for FAKE, RELIABLE AND CONSPIRACY data. 
# Run the rogram with EXACTLY 5 argruments as shown below:
#       python <script> <input-file-name> <output-CSV-file> <#skipInitialRows> <#rowsToProcess> <#rowsAfterWhichToShowPrintMessageTracker>
#
#       #skipInitialRows
#               is an integer. Is the CSV data row number (0 being for header) from which the data
#               should be loaded. Here 0 is the header so to skip first 5 data rows: skipInitialRows = 5.
#       #rowsToProcess
#               is an integer. If = -1 then will read in all the rows after skipping as per above parameter.
#               Otherwise, will read in the number specified.
## Note only if running in jupyter then uncomment the two lines for import nest_asyncio and the nest_asyncio.apply()
#
# The packages expected by the script are:
# import pandas as pd
# import twint
# import csv
# #import os
# import sys
# from datetime import datetime
# import logging

In [5]:
!python /home/rohit/SRH/CaseStudy1/dataDemo/code/scriptExtractTweets1.py /home/rohit/SRH/CaseStudy1/dataDemo/dataFiles/demorun_input_data.csv /home/rohit/SRH/CaseStudy1/dataDemo/dataFiles/twint_data.csv 0 15 1
#output file: /home/rohit/SRH/CaseStudy1/dataDemo/dataFiles/twint_data.csv

All command line arguments are valid....starting main processing

Start time: Tue Oct 22 10:34:55 2019


Processing with command line arguments as:
1) dataIpFile       = /home/rohit/SRH/CaseStudy1/dataDemo/dataFiles/demorun_input_data.csv
2) csvOpFile        = /home/rohit/SRH/CaseStudy1/dataDemo/dataFiles/twint_data.csv
3) skipInitialRows  = 0
4) rowsToProcess    = 15
5) printMessageFreq = 1


Input data read into dataframe.


Processing row number 1
Number of rows skipped   = 0
Number of rows processed = 1

End time: Tue Oct 22 10:34:56 2019


## Tweets have been extracted.

# T features -- extract TWITTER features

In [6]:
dataIpFile = '/home/rohit/SRH/CaseStudy1/dataDemo/dataFiles/twint_data.csv'
ipDataUsecolsList = [ 'date' , 'user_id_str' , 'nlikes' , 'nreplies' , 'nretweets' , 'search' ]

In [7]:
skipInitialRows = 0
dfIpData = pd.read_csv(dataIpFile, usecols = ipDataUsecolsList, skiprows = range(1, skipInitialRows + 1), sep = ',', low_memory=False)

In [8]:
dfIpData['domain'] = "garbage"

In [9]:
dfIpData['dateNew'] = pd.to_datetime(dfIpData['date'])
dfOut = dfIpData[['search', 'domain']]
dfOut = dfOut.drop_duplicates(subset = ['search'], keep = 'first')
dfOut = dfOut.reset_index(drop=True)
dfOut['urlType'] = "garbage"

####################################################
## number of tweets  (n)
dfIpTemp = dfIpData.copy()
dfIpTemp['totalTweets'] = dfIpTemp['search'].map(dfIpTemp['search'].value_counts())
dfIpTemp = dfIpTemp.drop_duplicates(subset=['search'], keep='first')
dfIpTemp = dfIpTemp.reset_index(drop=True)
dfOut['totalTweets'] = dfIpTemp['totalTweets']
del(dfIpTemp)

####################################################
## totalRetweets, totalLikes, totalReplies
dfIpTemp = dfIpData.copy()
dfIpTemp['totalRetweets'] = dfIpTemp.groupby('search')['nretweets'].transform('sum')
dfIpTemp = dfIpTemp.drop_duplicates(subset=['search'], keep='first')
dfIpTemp = dfIpTemp.reset_index(drop=True)
dfOut['totalRetweets'] =  dfIpTemp['totalRetweets']
del(dfIpTemp)
#
dfIpTemp = dfIpData.copy()
dfIpTemp['totalLikes'] = dfIpTemp.groupby('search')['nlikes'].transform('sum')
dfIpTemp = dfIpTemp.drop_duplicates(subset=['search'], keep='first')
dfIpTemp = dfIpTemp.reset_index(drop=True)
dfOut['totalLikes'] =  dfIpTemp['totalLikes']
del(dfIpTemp)
#
dfIpTemp = dfIpData.copy()
dfIpTemp['totalReplies'] = dfIpTemp.groupby('search')['nreplies'].transform('sum')
dfIpTemp = dfIpTemp.drop_duplicates(subset=['search'], keep='first')
dfIpTemp = dfIpTemp.reset_index(drop=True)
dfOut['totalReplies'] =  dfIpTemp['totalReplies']
del(dfIpTemp)
#

####################################################
## span
dfIpTemp = dfIpData.groupby('search', sort=False, as_index=False).apply(pd.DataFrame.sort_values, 'dateNew')
dfOutTemp = dfIpTemp.groupby('search')['dateNew'].agg(['min', 'max'])
del(dfIpTemp)
dfOutTemp.reset_index(inplace = True)
dfOutTemp['max'] = pd.to_datetime(dfOutTemp['max'])
dfOutTemp['min'] = pd.to_datetime(dfOutTemp['min'])
dfOutTemp['span'] = (dfOutTemp['max'] - dfOutTemp['min']) / np.timedelta64(1, 'h') 
dfOut.insert(7, 'span', dfOut['search'].map(dfOutTemp.set_index('search')['span']))
dfOut.loc[dfOut['span'] == 0, 'span'] = -1
del(dfOutTemp)

####################################################
# date processing to find out weekday, weekend, day of week counts
## process Weekday counts
dfIpTemp = dfIpData.copy()
#
dfIpTemp['tempDayOfWeek'] = dfIpTemp['dateNew'].dt.day_name()
#
dfIpTemp.loc[((dfIpTemp['tempDayOfWeek'] == 'Saturday') | (dfIpTemp['tempDayOfWeek'] == 'Sunday')), 'tempWkdayWkendType'] = 'Weekend'
dfIpTemp.loc[dfIpTemp['tempWkdayWkendType'] != 'Weekend', 'tempWkdayWkendType'] = 'Weekday'
#
dfIpTemp.loc[dfIpTemp['tempWkdayWkendType'] == 'Weekday', 'countWeekDay'] = 1
dfIpTemp['countWeekDay'] = dfIpTemp['countWeekDay'].fillna(0)
#
dfIpTemp['countWeekDay'] = dfIpTemp.groupby('search')['countWeekDay'].transform('sum')
dfIpTemp = dfIpTemp.drop_duplicates(subset=['search'], keep='first')
dfIpTemp = dfIpTemp.reset_index(drop=True)
#
dfOut['countWeekDay'] = dfIpTemp['countWeekDay']
#
del(dfIpTemp)
#
## process Weekend counts
dfIpTemp = dfIpData.copy()
#
dfIpTemp['tempDayOfWeek'] = dfIpTemp['dateNew'].dt.day_name()
#
dfIpTemp.loc[((dfIpTemp['tempDayOfWeek'] == 'Saturday') | (dfIpTemp['tempDayOfWeek'] == 'Sunday')), 'tempWkdayWkendType'] = 'Weekend'
dfIpTemp.loc[dfIpTemp['tempWkdayWkendType'] != 'Weekend', 'tempWkdayWkendType'] = 'Weekday'
#
dfIpTemp.loc[dfIpTemp['tempWkdayWkendType'] == 'Weekend', 'countWeekEnd'] = 1
dfIpTemp['countWeekEnd'] = dfIpTemp['countWeekEnd'].fillna(0)
#
dfIpTemp['countWeekEnd']   = dfIpTemp.groupby('search')['countWeekEnd'].transform('sum')
dfIpTemp = dfIpTemp.drop_duplicates(subset=['search'], keep='first')
dfIpTemp = dfIpTemp.reset_index(drop=True)
#
dfOut['countWeekEnd'] = dfIpTemp['countWeekEnd']
#
del(dfIpTemp)
#
## process Monday counts
dfIpTemp = dfIpData.copy()
#
dfIpTemp['tempDayOfWeek'] = dfIpTemp['dateNew'].dt.day_name()
#
dfIpTemp.loc[dfIpTemp['tempDayOfWeek'] == 'Monday', 'countMonday'] = 1
dfIpTemp['countMonday'] = dfIpTemp['countMonday'].fillna(0)
#
dfIpTemp['countMonday'] = dfIpTemp.groupby('search')['countMonday'].transform('sum')
dfIpTemp = dfIpTemp.drop_duplicates(subset=['search'], keep='first')
dfIpTemp = dfIpTemp.reset_index(drop=True)
#
dfOut['countMonday'] = dfIpTemp['countMonday']
#
del(dfIpTemp)
#
## process Tuesday counts
dfIpTemp = dfIpData.copy()
#
dfIpTemp['tempDayOfWeek'] = dfIpTemp['dateNew'].dt.day_name()
#
dfIpTemp.loc[dfIpTemp['tempDayOfWeek'] == 'Tuesday', 'countTuesday'] = 1
dfIpTemp['countTuesday'] = dfIpTemp['countTuesday'].fillna(0)
#
dfIpTemp['countTuesday'] = dfIpTemp.groupby('search')['countTuesday'].transform('sum')
dfIpTemp = dfIpTemp.drop_duplicates(subset=['search'], keep='first')
dfIpTemp = dfIpTemp.reset_index(drop=True)
#
dfOut['countTuesday'] = dfIpTemp['countTuesday']
#
del(dfIpTemp)
#
## process Wednesday counts
dfIpTemp = dfIpData.copy()
#
dfIpTemp['tempDayOfWeek'] = dfIpTemp['dateNew'].dt.day_name()
#
dfIpTemp.loc[dfIpTemp['tempDayOfWeek'] == 'Wednesday', 'countWednesday'] = 1
dfIpTemp['countWednesday'] = dfIpTemp['countWednesday'].fillna(0)
#
dfIpTemp['countWednesday'] = dfIpTemp.groupby('search')['countWednesday'].transform('sum')
dfIpTemp = dfIpTemp.drop_duplicates(subset=['search'], keep='first')
dfIpTemp = dfIpTemp.reset_index(drop=True)
#
dfOut['countWednesday'] = dfIpTemp['countWednesday']
#
del(dfIpTemp)
#
## process Thursday counts
dfIpTemp = dfIpData.copy()
#
dfIpTemp['tempDayOfWeek'] = dfIpTemp['dateNew'].dt.day_name()
#
dfIpTemp.loc[dfIpTemp['tempDayOfWeek'] == 'Thursday', 'countThursday'] = 1
dfIpTemp['countThursday'] = dfIpTemp['countThursday'].fillna(0)
#
dfIpTemp['countThursday'] = dfIpTemp.groupby('search')['countThursday'].transform('sum')
dfIpTemp = dfIpTemp.drop_duplicates(subset=['search'], keep='first')
dfIpTemp = dfIpTemp.reset_index(drop=True)
#
dfOut['countThursday'] = dfIpTemp['countThursday']
#
del(dfIpTemp)
#
## process Friday counts
dfIpTemp = dfIpData.copy()
#
dfIpTemp['tempDayOfWeek'] = dfIpTemp['dateNew'].dt.day_name()
#
dfIpTemp.loc[dfIpTemp['tempDayOfWeek'] == 'Friday', 'countFriday'] = 1
dfIpTemp['countFriday'] = dfIpTemp['countFriday'].fillna(0)
#
dfIpTemp['countFriday'] = dfIpTemp.groupby('search')['countFriday'].transform('sum')
dfIpTemp = dfIpTemp.drop_duplicates(subset=['search'], keep='first')
dfIpTemp = dfIpTemp.reset_index(drop=True)
#
dfOut['countFriday'] = dfIpTemp['countFriday']
#
del(dfIpTemp)
#
## process Saturday counts
dfIpTemp = dfIpData.copy()
#
dfIpTemp['tempDayOfWeek'] = dfIpTemp['dateNew'].dt.day_name()
#
dfIpTemp.loc[dfIpTemp['tempDayOfWeek'] == 'Saturday', 'countSaturday'] = 1
dfIpTemp['countSaturday'] = dfIpTemp['countSaturday'].fillna(0)
#
dfIpTemp['countSaturday'] = dfIpTemp.groupby('search')['countSaturday'].transform('sum')
dfIpTemp = dfIpTemp.drop_duplicates(subset=['search'], keep='first')
dfIpTemp = dfIpTemp.reset_index(drop=True)
#
dfOut['countSaturday'] = dfIpTemp['countSaturday']
#
del(dfIpTemp)
#
## process Sunday counts
dfIpTemp = dfIpData.copy()
#
dfIpTemp['tempDayOfWeek'] = dfIpTemp['dateNew'].dt.day_name()
#
dfIpTemp.loc[dfIpTemp['tempDayOfWeek'] == 'Sunday', 'countSunday'] = 1
dfIpTemp['countSunday'] = dfIpTemp['countSunday'].fillna(0)
#
dfIpTemp['countSunday'] = dfIpTemp.groupby('search')['countSunday'].transform('sum')
dfIpTemp = dfIpTemp.drop_duplicates(subset=['search'], keep='first')
dfIpTemp = dfIpTemp.reset_index(drop=True)
#
dfOut['countSunday'] = dfIpTemp['countSunday']
#
del(dfIpTemp)
#

####################################################
## average time between tweets
dfIpTemp = dfIpData.groupby('search', sort=False, as_index=False).apply(pd.DataFrame.sort_values, 'dateNew')
dfIpTemp['timeBetween2Tweets'] = dfIpTemp.groupby('search')['dateNew'].diff().fillna(0)
dfIpTemp['timeBetween2Tweets'] = dfIpTemp['timeBetween2Tweets'] / np.timedelta64(1, 'h')
## using sum and dividing by n-1
dfOutTemp = dfIpData[['search']]
dfIpTemp = dfIpTemp.reset_index(drop=True)
dfOutTemp['avgTimeBetweenTweets'] = dfIpTemp.groupby('search')['timeBetween2Tweets'].transform('sum')
dfOutTemp['countTweet'] = dfOutTemp['search'].map(dfOutTemp['search'].value_counts())
dfOutTemp = dfOutTemp.drop_duplicates(subset=['search'], keep='first')
dfOutTemp = dfOutTemp.reset_index(drop=True)
dfOutTemp['avgTimeBetweenTweets'] = dfOutTemp['avgTimeBetweenTweets'] / (dfOutTemp['countTweet'] - 1)
dfOutTemp.loc[dfOutTemp['avgTimeBetweenTweets'].isnull(), 'avgTimeBetweenTweets'] = -1
dfOut['avgTimeBetweenTweets'] = dfOutTemp['avgTimeBetweenTweets']
del(dfOutTemp)
del(dfIpTemp)

####################################################
## average time of next tweet
dfIpTemp = dfIpData.groupby('search', sort=False, as_index=False).apply(pd.DataFrame.sort_values, 'dateNew')
dfIpTemp['timeFromFirstTweet'] = dfIpTemp['dateNew'] - dfIpTemp.groupby('search')['dateNew'].transform('first')
dfIpTemp['timeFromFirstTweet'] = dfIpTemp['timeFromFirstTweet']/np.timedelta64(1, 'h')
dfIpTemp['countTweets'] = dfIpTemp['search'].map(dfIpTemp['search'].value_counts())
dfIpTemp['avgTimeOfNextTweet'] = dfIpTemp.groupby('search')['timeFromFirstTweet'].transform(sum)
dfIpTemp['avgTimeOfNextTweet'] = dfIpTemp['avgTimeOfNextTweet']/(dfIpTemp['countTweets'] - 1)
dfIpTemp = dfIpTemp.drop_duplicates(subset = ['search'], keep = 'first')
dfIpTemp.loc[dfIpTemp['avgTimeOfNextTweet'].isnull(), 'avgTimeOfNextTweet'] = -1
dfIpTemp = dfIpTemp.reset_index(drop=True)
dfOut = dfOut.reset_index(drop=True)
dfOut['avgTimeOfNextTweet'] = dfIpTemp['avgTimeOfNextTweet']
del(dfIpTemp)

####################################################
## Time Absolute Bins
dfIpTemp = dfIpData.groupby('search', sort=False, as_index=False).apply(pd.DataFrame.sort_values, 'dateNew')
dfIpTemp['timeFromFirstTweet'] = dfIpTemp['dateNew'] - dfIpTemp.groupby('search')['dateNew'].transform('first')
dfIpTemp['timeFromFirstTweet'] = dfIpTemp['timeFromFirstTweet']/np.timedelta64(1, 'h')
dfIpTemp['Bins'] = ""
Bins = []
for time in dfIpTemp['timeFromFirstTweet']:
    if time <= 6:
        Bins.append('0 - 6')
    elif time > 6 and time <= 12:
        Bins.append('6 - 12')
    elif time > 12 and time <= 18:
        Bins.append('12 - 18')
    elif time > 18 and time <= 24:
        Bins.append('18 - 24')
    else:
        Bins.append('Greater than 24')
dfIpTemp['Bins'] = Bins
dfIpTemp.loc[dfIpTemp['Bins'] == '0 - 6', '0 - 6 hour bin'] = 1
dfIpTemp['0 - 6 hour bin'] = dfIpTemp['0 - 6 hour bin'].fillna(0)
#
dfIpTemp.loc[dfIpTemp['Bins'] == '6 - 12', '6 - 12 hour bin'] = 1
dfIpTemp['6 - 12 hour bin'] = dfIpTemp['6 - 12 hour bin'].fillna(0)
#
dfIpTemp.loc[dfIpTemp['Bins'] == '12 - 18', '12 - 18 hour bin'] = 1
dfIpTemp['12 - 18 hour bin'] = dfIpTemp['12 - 18 hour bin'].fillna(0)
#
dfIpTemp.loc[dfIpTemp['Bins'] == '18 - 24', '18 - 24 hour bin'] = 1
dfIpTemp['18 - 24 hour bin'] = dfIpTemp['18 - 24 hour bin'].fillna(0)
#
dfIpTemp.loc[dfIpTemp['Bins'] == 'Greater than 24', 'Greater than 24'] = 1
dfIpTemp['Greater than 24'] = dfIpTemp['Greater than 24'].fillna(0)
#
dfIpTemp['count0to6'] = dfIpTemp.groupby('search')['0 - 6 hour bin'].transform('sum')
dfIpTemp['count6to12'] = dfIpTemp.groupby('search')['6 - 12 hour bin'].transform('sum')
dfIpTemp['count12to18'] = dfIpTemp.groupby('search')['12 - 18 hour bin'].transform('sum')
dfIpTemp['count18to24'] = dfIpTemp.groupby('search')['18 - 24 hour bin'].transform('sum')
dfIpTemp['count24plus'] = dfIpTemp.groupby('search')['Greater than 24'].transform('sum')
#
dfIpTemp = dfIpTemp.reset_index(drop=True)
dfIpTemp = dfIpTemp.drop_duplicates(subset=['search'], keep='first')
dfIpTemp = dfIpTemp.reset_index(drop=True)
#
dfOut['timeAbsBin1count0to6'] = dfIpTemp['count0to6']
dfOut['timeAbsBin2count6to12'] = dfIpTemp['count6to12']
dfOut['timeAbsBin3count12to18'] = dfIpTemp['count12to18']
dfOut['timeAbsBin4count18to24'] = dfIpTemp['count18to24']
dfOut['timeAbsBin5count24plus'] = dfIpTemp['count24plus']
#
del(dfIpTemp)

####################################################
## avg Tweets Per Unique User
dfIpTemp = dfIpData[['date', 'user_id_str', 'search', 'dateNew']].copy()
dfIpTemp['user_id_str'] = dfIpTemp['user_id_str'].astype(str)
dfIpTemp['search_user_id_str'] = dfIpTemp['search'] + '___' + dfIpTemp['user_id_str']
dfIpTemp['countTweetsForUrl'] = dfIpTemp['search'].map(dfIpTemp['search'].value_counts())
dfIpTemp['countTweetsPerUser'] = dfIpTemp['search_user_id_str'].map(dfIpTemp['search_user_id_str'].value_counts())
dfIpTemp = dfIpTemp.drop_duplicates(subset = ['search_user_id_str'], keep = 'first')
dfIpTemp['countUniqueUsersPerUrl'] = dfIpTemp['search'].map(dfIpTemp['search'].value_counts())
del dfIpTemp['search_user_id_str']
dfIpTemp['avgTweetsPerUniqUser'] = dfIpTemp['countTweetsForUrl'] / dfIpTemp['countUniqueUsersPerUrl']
dfIpTemp = dfIpTemp.drop_duplicates(subset = ['search'], keep = 'first')
dfIpTemp = dfIpTemp.reset_index(drop=True)
dfOut['avgTweetsPerUniqUser'] = dfIpTemp['avgTweetsPerUniqUser']
#
del(dfIpTemp)

####################################################
####################################################
## normalize the features
scaler = MinMaxScaler()
dfOutTempNorm = dfOut[['totalTweets', 'totalRetweets', 'totalLikes', 'totalReplies',           \
                    'countWeekDay', 'countWeekEnd', 'countMonday', 'countTuesday',             \
                    'countWednesday', 'countThursday', 'countFriday', 'countSaturday',         \
                    'countSunday', 'avgTweetsPerUniqUser']]
dfOutTempNorm[['totalTweets', 'totalRetweets', 'totalLikes', 'totalReplies',                     \
                    'countWeekDay', 'countWeekEnd', 'countMonday', 'countTuesday',         \
                    'countWednesday', 'countThursday', 'countFriday', 'countSaturday',             \
                    'countSunday', 'avgTweetsPerUniqUser']]   \
= \
scaler.fit_transform(dfOutTempNorm[['totalTweets', 'totalRetweets', 'totalLikes', 'totalReplies', \
                    'countWeekDay', 'countWeekEnd', 'countMonday', 'countTuesday',  \
                    'countWednesday', 'countThursday', 'countFriday', 'countSaturday',      \
                    'countSunday', 'avgTweetsPerUniqUser']])
#
dfOut['normTotalTweets'] = dfOutTempNorm['totalTweets']
dfOut['normTotalRetweets'] = dfOutTempNorm['totalRetweets']
dfOut['normTotalLikes'] = dfOutTempNorm['totalLikes']
dfOut['normTotalReplies'] = dfOutTempNorm['totalReplies']
dfOut['normCountWeekDay'] = dfOutTempNorm['countWeekDay']
dfOut['normCountWeekEnd'] = dfOutTempNorm['countWeekEnd']
dfOut['normCountMonday'] = dfOutTempNorm['countMonday']
dfOut['normCountTuesday'] = dfOutTempNorm['countTuesday']
dfOut['normCountWednesday'] = dfOutTempNorm['countWednesday']
dfOut['normCountThursday'] = dfOutTempNorm['countThursday']
dfOut['normCountFriday'] = dfOutTempNorm['countFriday']
dfOut['normCountSaturday'] = dfOutTempNorm['countSaturday']
dfOut['normCountSunday'] = dfOutTempNorm['countSunday']
dfOut['normAvgTweetsPerUniqUser'] = dfOutTempNorm['avgTweetsPerUniqUser']
#
del(dfOutTempNorm)
#
dfBinNormTemp = dfOut[['timeAbsBin1count0to6', 'timeAbsBin2count6to12',     \
                       'timeAbsBin3count12to18', 'timeAbsBin4count18to24',  \
                        'timeAbsBin5count24plus']]
#
dfBinNormTemp['totalTimeAbsBin5count']   =      \
dfBinNormTemp['timeAbsBin1count0to6']    +      \
dfBinNormTemp['timeAbsBin2count6to12']   +      \
dfBinNormTemp['timeAbsBin3count12to18']  +      \
dfBinNormTemp['timeAbsBin4count18to24']  +      \
dfBinNormTemp['timeAbsBin5count24plus']
#
dfBinNormTemp['NORMtimeAbsBin1count0to6']   = dfBinNormTemp['timeAbsBin1count0to6']    / dfBinNormTemp['totalTimeAbsBin5count']
dfBinNormTemp['NORMtimeAbsBin1count6to12']  = dfBinNormTemp['timeAbsBin2count6to12']   / dfBinNormTemp['totalTimeAbsBin5count']
dfBinNormTemp['NORMtimeAbsBin1count12to18'] = dfBinNormTemp['timeAbsBin3count12to18']  / dfBinNormTemp['totalTimeAbsBin5count']
dfBinNormTemp['NORMtimeAbsBin1count18to24'] = dfBinNormTemp['timeAbsBin4count18to24']  / dfBinNormTemp['totalTimeAbsBin5count']
dfBinNormTemp['NORMtimeAbsBin5count24plus'] = dfBinNormTemp['timeAbsBin5count24plus']  / dfBinNormTemp['totalTimeAbsBin5count']
#
dfOut['normTimeAbsBin1count0to6']   = dfBinNormTemp['NORMtimeAbsBin1count0to6']
dfOut['normTimeAbsBin2count6to12']  = dfBinNormTemp['NORMtimeAbsBin1count6to12']
dfOut['normTimeAbsBin3count12to18'] = dfBinNormTemp['NORMtimeAbsBin1count12to18']
dfOut['normTimeAbsBin4count18to24'] = dfBinNormTemp['NORMtimeAbsBin1count18to24']
dfOut['normTimeAbsBin5count24plus'] = dfBinNormTemp['NORMtimeAbsBin5count24plus']
#
del(dfBinNormTemp)
#
## normalise the span, avgTimeBetweenTweets, avgTimeOfNextTweet by first setting the -1 values to 0,
#        applying min-max, then reinserting the -1 at the same positions as before
dfOutTemp = dfOut[['span', 'avgTimeBetweenTweets', 'avgTimeOfNextTweet']]
dfOutTemp['normSpan']                 = dfOutTemp['span']
dfOutTemp['normAvgTimeBetweenTweets'] = dfOutTemp['avgTimeBetweenTweets']
dfOutTemp['normAvgTimeOfNextTweet']   = dfOutTemp['avgTimeOfNextTweet']
dfOutTemp['normSpan']                 = dfOutTemp['normSpan'].replace(-1, 0)
dfOutTemp['normAvgTimeBetweenTweets'] = dfOutTemp['normAvgTimeBetweenTweets'].replace(-1, 0)
dfOutTemp['normAvgTimeOfNextTweet']   = dfOutTemp['normAvgTimeOfNextTweet'].replace(-1, 0)
dfOutTemp[['normSpan', 'normAvgTimeBetweenTweets', 'normAvgTimeOfNextTweet']] =            \
scaler.fit_transform(dfOutTemp[['normSpan', 'normAvgTimeBetweenTweets', 'normAvgTimeOfNextTweet']])
dfOutTemp.loc[dfOutTemp['span'] == -1 , 'normSpan'] = -1
dfOutTemp.loc[dfOutTemp['avgTimeBetweenTweets'] == -1 , 'normAvgTimeBetweenTweets'] = -1
dfOutTemp.loc[dfOutTemp['avgTimeOfNextTweet'] == -1 , 'normAvgTimeOfNextTweet'] = -1
dfOut['normSpan']                 = dfOutTemp['normSpan']
dfOut['normAvgTimeBetweenTweets'] = dfOutTemp['normAvgTimeBetweenTweets']
dfOut['normAvgTimeOfNextTweet']   = dfOutTemp['normAvgTimeOfNextTweet']
#
del(dfOutTemp)
dfOut.rename(columns = {'search':'url'}, inplace = True)
#
## creating these additional fields which can be used for some data visualisation later
dfOut['normPercentWeekDayByTotalTweets']   = dfOut['countWeekDay']   / dfOut['totalTweets']
dfOut['normPercentWeekEndByTotalTweets']   = dfOut['countWeekEnd']   / dfOut['totalTweets']
dfOut['normPercentMondayByTotalTweets']    = dfOut['countMonday']    / dfOut['totalTweets']
dfOut['normPercentTuesdayByTotalTweets']   = dfOut['countTuesday']   / dfOut['totalTweets']
dfOut['normPercentWednesdayByTotalTweets'] = dfOut['countWednesday'] / dfOut['totalTweets']
dfOut['normPercentThursdayByTotalTweets']  = dfOut['countThursday']  / dfOut['totalTweets']
dfOut['normPercentFridayByTotalTweets']    = dfOut['countFriday']    / dfOut['totalTweets']
dfOut['normPercentSaturdayByTotalTweets']  = dfOut['countSaturday']  / dfOut['totalTweets']
dfOut['normPercentSundayByTotalTweets']    = dfOut['countSunday']    / dfOut['totalTweets']
#
# deleting the urlType as it is garbage values
del dfOut['urlType']
del dfOut['domain']

/home/rohit/anaconda3/envs/fakenewsdemo1/lib/python3.6/site-packages/ipykernel_launcher.py:209: FutureWarning: Passing integers to fillna is deprecated, will raise a TypeError in a future version.  To retain the old behavior, pass pd.Timedelta(seconds=n) instead.
/home/rohit/anaconda3/envs/fakenewsdemo1/lib/python3.6/site-packages/ipykernel_launcher.py:214: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/rohit/anaconda3/envs/fakenewsdemo1/lib/python3.6/site-packages/ipykernel_launcher.py:215: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

# writing the twitter features file

In [10]:
opFileDfOutWITHNormValue = '/home/rohit/SRH/CaseStudy1/dataDemo/dataFiles/temp_T_features.csv'
dfOut.to_csv(opFileDfOutWITHNormValue, index=False)
#
del dfOut
del dfIpData
#
### finished

# M features -- extract MOPHOLOGICAL features (using spaCy) -- using the title+content

In [16]:
ipDataFile = urlsFile4Demo

In [17]:
skipInitialRows = 0
data = pd.read_csv(ipDataFile, skiprows = range(1, skipInitialRows + 1), sep = ',', low_memory=False)

In [18]:
####################################################
####################################################
## define the functions to be used

def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text()
    return stripped_text

#strip_html_tags('<html><h2>Some important text</h2></html>')
#
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

#remove_accented_chars('Sómě Áccěntěd těxt')
#
def expand_contractions(text, contractions_dict):
    contractions_pattern = re.compile('({})'.format('|'.join(contractions_dict.keys())),
                                      flags=re.IGNORECASE | re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        
        expanded_contraction = contractions_dict.get(match) \
            if contractions_dict.get(match) \
            else contractions_dict.get(match.lower())
        expanded_contraction = expanded_contraction
        
        return expanded_contraction
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    expanded_text = "%s%s" % (expanded_text[0].upper(), expanded_text[1:])

    return expanded_text

#expand_contractions("You all can't expand contractions I'd think", contractions_dict)
#
def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

#remove_special_characters("Well this was fun! What do you think? 123#@!", remove_digits=False)
#
def lemmatize_text(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

#lemmatize_text("My system keeps crashing! his crashed yesterday, ours crashes daily")
#
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

#remove_stopwords("The, and, if are stopwords, computer is not")
#
def normalize_corpus(corpus, html_stripping=True, contraction_expansion=True,
                     accented_char_removal=True, text_lower_case=False, 
                     text_lemmatization=False, special_char_removal=False, 
                     stopword_removal=True, remove_digits=True):
    
    normalized_corpus = []
    # normalize each document in the corpus
    for doc in corpus:
        # strip HTML
        if html_stripping:
            doc = strip_html_tags(doc)
        # remove accented characters
        if accented_char_removal:
            doc = remove_accented_chars(doc)
        # expand contractions    
        if contraction_expansion:
            doc = expand_contractions(doc, contractions_dict)
        # lowercase the text    
        if text_lower_case:
            doc = doc.lower()
        # remove extra newlines
        doc = re.sub(r'[\r|\n|\r\n]+', ' ',doc)
        # lemmatize text
        if text_lemmatization:
            doc = lemmatize_text(doc)
        # remove special characters and\or digits    
        if special_char_removal:
            # insert spaces between special characters to isolate them    
            special_char_pattern = re.compile(r'([{.(-)!}])')
            doc = special_char_pattern.sub(" \\1 ", doc)
            doc = remove_special_characters(doc, remove_digits=remove_digits)  
        # remove extra whitespace
        doc = re.sub(' +', ' ', doc)
        # remove stopwords
        if stopword_removal:
            doc = remove_stopwords(doc, is_lower_case=text_lower_case)
            
        normalized_corpus.append(doc)
        
    return normalized_corpus
#
tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')
nlp = en_core_web_lg.load()
#
data['full_text'] = data["title"].map(str)+ '. ' + data["content"]
del data['title']
del data['content']
#
data['clean_full_text'] = normalize_corpus(data['full_text'])
del data['full_text']
#
dfOut = data.copy()
## there are 50 features
new_cols = ['$', "''", ',', '-LRB-', '-RRB-', '.', ':', 'ADD', 'AFX', 'CC', 'CD', 'DT', 'EX', 'FW', 'HYPH', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NFP', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', 'XX', '_SP', '``']
dfOut = dfOut.assign(**dict.fromkeys(new_cols, -1.00))
## no need to keep the data dataframe anymore
del(data)
#

In [19]:
printFreqValue = 1
# POS tagging for sample cleaned full text
l = 0
#mf_clean_full_text = data3
while l < len(dfOut):
    sentence = str(dfOut.iloc[l].clean_full_text)
    sentence_nlp = nlp(sentence)
    # POS tagging with Spacy 
    spacy_pos_tagged = [(word, word.tag_, word.pos_) for word in sentence_nlp]
    postag_data = pd.DataFrame(spacy_pos_tagged, columns=['Word', 'POS tag', 'Tag type'])
    mb = postag_data.groupby(['POS tag']).count()
    mc = mb.drop(['Tag type'], axis = 1)
    mc = mc.rename_axis("", axis="rows").transpose()
    mc.index = [l]
    dfOut.update(mc)
    l=l+1
    if l % printFreqValue == 0:
        print(f"Finished row number = {l - 1}\nfor url = {dfOut.iloc[l-1].url}")
#
del dfOut['clean_full_text']
print(f"\nDone")

Finished row number = 0
for url = http://beforeitsnews.com/blogging-citizen-journalism/2018/01/attention-forward-to-president-trump-bombshell-breadcrumb-from-judge-anna-von-reitz-urgent-public-message-2585490.html

Done


# renaming 6 columns to more meaningful names

In [20]:
######     The six columns, old and new names
#
## $       dollar
## ''      dblApostrophe
## ,       dblApostrophe
## .       dot
## :       colon
## ``      dblSpecialApostrophe
#
dfOut.rename(columns = { '$'  : 'dollar'},               inplace = True)
dfOut.rename(columns = { "''" : 'dblApostrophe'},        inplace = True)
dfOut.rename(columns = { ','  : 'comma'},                inplace = True)
dfOut.rename(columns = { '.'  : 'dot'},                  inplace = True)
dfOut.rename(columns = { ':'  : 'colon'},                inplace = True)
dfOut.rename(columns = { '``' : 'dblSpecialApostrophe'}, inplace = True)

In [21]:
####################################################
####################################################
## normalize the features
scaler = MinMaxScaler()
dfOutTempNorm = dfOut.copy()
del dfOutTempNorm['domain']
del dfOutTempNorm['url']
#
dfOutTempNorm['normDollar']               = dfOutTempNorm['dollar']
dfOutTempNorm['normDblApostrophe']        = dfOutTempNorm['dblApostrophe']
dfOutTempNorm['normComma']                = dfOutTempNorm['comma']
dfOutTempNorm['norm-LRB-']                = dfOutTempNorm['-LRB-']
dfOutTempNorm['norm-RRB-']                = dfOutTempNorm['-RRB-']
dfOutTempNorm['normDot']                  = dfOutTempNorm['dot']
dfOutTempNorm['normColon']                = dfOutTempNorm['colon']
dfOutTempNorm['normADD']                  = dfOutTempNorm['ADD']
dfOutTempNorm['normAFX']                  = dfOutTempNorm['AFX']
dfOutTempNorm['normCC']                   = dfOutTempNorm['CC']
dfOutTempNorm['normCD']                   = dfOutTempNorm['CD']
dfOutTempNorm['normDT']                   = dfOutTempNorm['DT']
dfOutTempNorm['normEX']                   = dfOutTempNorm['EX']
dfOutTempNorm['normFW']                   = dfOutTempNorm['FW']
dfOutTempNorm['normHYPH']                 = dfOutTempNorm['HYPH']
dfOutTempNorm['normIN']                   = dfOutTempNorm['IN']
dfOutTempNorm['normJJ']                   = dfOutTempNorm['JJ']
dfOutTempNorm['normJJR']                  = dfOutTempNorm['JJR']
dfOutTempNorm['normJJS']                  = dfOutTempNorm['JJS']
dfOutTempNorm['normLS']                   = dfOutTempNorm['LS']
dfOutTempNorm['normMD']                   = dfOutTempNorm['MD']
dfOutTempNorm['normNFP']                  = dfOutTempNorm['NFP']
dfOutTempNorm['normNN']                   = dfOutTempNorm['NN']
dfOutTempNorm['normNNP']                  = dfOutTempNorm['NNP']
dfOutTempNorm['normNNPS']                 = dfOutTempNorm['NNPS']
dfOutTempNorm['normNNS']                  = dfOutTempNorm['NNS']
dfOutTempNorm['normPDT']                  = dfOutTempNorm['PDT']
dfOutTempNorm['normPOS']                  = dfOutTempNorm['POS']
dfOutTempNorm['normPRP']                  = dfOutTempNorm['PRP']
dfOutTempNorm['normPRP$']                 = dfOutTempNorm['PRP$']
dfOutTempNorm['normRB']                   = dfOutTempNorm['RB']
dfOutTempNorm['normRBR']                  = dfOutTempNorm['RBR']
dfOutTempNorm['normRBS']                  = dfOutTempNorm['RBS']
dfOutTempNorm['normRP']                   = dfOutTempNorm['RP']
dfOutTempNorm['normSYM']                  = dfOutTempNorm['SYM']
dfOutTempNorm['normTO']                   = dfOutTempNorm['TO']
dfOutTempNorm['normUH']                   = dfOutTempNorm['UH']
dfOutTempNorm['normVB']                   = dfOutTempNorm['VB']
dfOutTempNorm['normVBD']                  = dfOutTempNorm['VBD']
dfOutTempNorm['normVBG']                  = dfOutTempNorm['VBG']
dfOutTempNorm['normVBN']                  = dfOutTempNorm['VBN']
dfOutTempNorm['normVBP']                  = dfOutTempNorm['VBP']
dfOutTempNorm['normVBZ']                  = dfOutTempNorm['VBZ']
dfOutTempNorm['normWDT']                  = dfOutTempNorm['WDT']
dfOutTempNorm['normWP']                   = dfOutTempNorm['WP']
dfOutTempNorm['normWP$']                  = dfOutTempNorm['WP$']
dfOutTempNorm['normWRB']                  = dfOutTempNorm['WRB']
dfOutTempNorm['normXX']                   = dfOutTempNorm['XX']
dfOutTempNorm['norm_SP']                  = dfOutTempNorm['_SP']
dfOutTempNorm['normDblSpecialApostrophe'] = dfOutTempNorm['dblSpecialApostrophe']
#
dfOutTempNorm['normDollar']               = dfOutTempNorm['normDollar'].replace(-1, 0)
dfOutTempNorm['normDblApostrophe']        = dfOutTempNorm['normDblApostrophe'].replace(-1, 0)
dfOutTempNorm['normComma']                = dfOutTempNorm['normComma'].replace(-1, 0)
dfOutTempNorm['norm-LRB-']                = dfOutTempNorm['norm-LRB-'].replace(-1, 0)
dfOutTempNorm['norm-RRB-']                = dfOutTempNorm['norm-RRB-'].replace(-1, 0)
dfOutTempNorm['normDot']                  = dfOutTempNorm['normDot'].replace(-1, 0)
dfOutTempNorm['normColon']                = dfOutTempNorm['normColon'].replace(-1, 0)
dfOutTempNorm['normADD']                  = dfOutTempNorm['normADD'].replace(-1, 0)
dfOutTempNorm['normAFX']                  = dfOutTempNorm['normAFX'].replace(-1, 0)
dfOutTempNorm['normCC']                   = dfOutTempNorm['normCC'].replace(-1, 0)
dfOutTempNorm['normCD']                   = dfOutTempNorm['normCD'].replace(-1, 0)
dfOutTempNorm['normDT']                   = dfOutTempNorm['normDT'].replace(-1, 0)
dfOutTempNorm['normEX']                   = dfOutTempNorm['normEX'].replace(-1, 0)
dfOutTempNorm['normFW']                   = dfOutTempNorm['normFW'].replace(-1, 0)
dfOutTempNorm['normHYPH']                 = dfOutTempNorm['normHYPH'].replace(-1, 0)
dfOutTempNorm['normIN']                   = dfOutTempNorm['normIN'].replace(-1, 0)
dfOutTempNorm['normJJ']                   = dfOutTempNorm['normJJ'].replace(-1, 0)
dfOutTempNorm['normJJR']                  = dfOutTempNorm['normJJR'].replace(-1, 0)
dfOutTempNorm['normJJS']                  = dfOutTempNorm['normJJS'].replace(-1, 0)
dfOutTempNorm['normLS']                   = dfOutTempNorm['normLS'].replace(-1, 0)
dfOutTempNorm['normMD']                   = dfOutTempNorm['normMD'].replace(-1, 0)
dfOutTempNorm['normNFP']                  = dfOutTempNorm['normNFP'].replace(-1, 0)
dfOutTempNorm['normNN']                   = dfOutTempNorm['normNN'].replace(-1, 0)
dfOutTempNorm['normNNPS']                 = dfOutTempNorm['normNNPS'].replace(-1, 0)
dfOutTempNorm['normNNS']                  = dfOutTempNorm['normNNS'].replace(-1, 0)
dfOutTempNorm['normPDT']                  = dfOutTempNorm['normPDT'].replace(-1, 0)
dfOutTempNorm['normPOS']                  = dfOutTempNorm['normPOS'].replace(-1, 0)
dfOutTempNorm['normPRP']                  = dfOutTempNorm['normPRP'].replace(-1, 0)
dfOutTempNorm['normPRP$']                 = dfOutTempNorm['normPRP$'].replace(-1, 0)
dfOutTempNorm['normRB']                   = dfOutTempNorm['normRB'].replace(-1, 0)
dfOutTempNorm['normRBR']                  = dfOutTempNorm['normRBR'].replace(-1, 0)
dfOutTempNorm['normRBS']                  = dfOutTempNorm['normRBS'].replace(-1, 0)
dfOutTempNorm['normRP']                   = dfOutTempNorm['normRP'].replace(-1, 0)
dfOutTempNorm['normSYM']                  = dfOutTempNorm['normSYM'].replace(-1, 0)
dfOutTempNorm['normTO']                   = dfOutTempNorm['normTO'].replace(-1, 0)
dfOutTempNorm['normUH']                   = dfOutTempNorm['normUH'].replace(-1, 0)
dfOutTempNorm['normVB']                   = dfOutTempNorm['normVB'].replace(-1, 0)
dfOutTempNorm['normVBD']                  = dfOutTempNorm['normVBD'].replace(-1, 0)
dfOutTempNorm['normVBG']                  = dfOutTempNorm['normVBG'].replace(-1, 0)
dfOutTempNorm['normVBN']                  = dfOutTempNorm['normVBN'].replace(-1, 0)
dfOutTempNorm['normVBP']                  = dfOutTempNorm['normVBP'].replace(-1, 0)
dfOutTempNorm['normVBZ']                  = dfOutTempNorm['normVBZ'].replace(-1, 0)
dfOutTempNorm['normWDT']                  = dfOutTempNorm['normWDT'].replace(-1, 0)
dfOutTempNorm['normWP']                   = dfOutTempNorm['normWP'].replace(-1, 0)
dfOutTempNorm['normWP$']                  = dfOutTempNorm['normWP$'].replace(-1, 0)
dfOutTempNorm['normWRB']                  = dfOutTempNorm['normWRB'].replace(-1, 0)
dfOutTempNorm['normXX']                   = dfOutTempNorm['normXX'].replace(-1, 0)
dfOutTempNorm['norm_SP']                  = dfOutTempNorm['norm_SP'].replace(-1, 0)
dfOutTempNorm['normDblSpecialApostrophe'] = dfOutTempNorm['normDblSpecialApostrophe'].replace(-1, 0)
#
dfOutTempNorm[['normDollar', 'normDblApostrophe', 'normComma', 'norm-LRB-',
 'norm-RRB-', 'normDot', 'normColon', 'normADD', 'normAFX', 'normCC', 'normCD',
 'normDT', 'normEX', 'normFW', 'normHYPH', 'normIN', 'normJJ', 'normJJR', 'normJJS',
 'normLS', 'normMD', 'normNFP', 'normNN', 'normNNP', 'normNNPS', 'normNNS', 'normPDT',
 'normPOS', 'normPRP', 'normPRP$', 'normRB', 'normRBR', 'normRBS', 'normRP',
 'normSYM', 'normTO', 'normUH', 'normVB', 'normVBD', 'normVBG', 'normVBN',
 'normVBP', 'normVBZ', 'normWDT', 'normWP', 'normWP$', 'normWRB',
 'normXX', 'norm_SP', 'normDblSpecialApostrophe']] = \
scaler.fit_transform(dfOutTempNorm[['normDollar', 'normDblApostrophe', 'normComma', 'norm-LRB-',
 'norm-RRB-', 'normDot', 'normColon', 'normADD', 'normAFX', 'normCC', 'normCD',
 'normDT', 'normEX', 'normFW', 'normHYPH', 'normIN', 'normJJ', 'normJJR', 'normJJS',
 'normLS', 'normMD', 'normNFP', 'normNN', 'normNNP', 'normNNPS', 'normNNS', 'normPDT',
 'normPOS', 'normPRP', 'normPRP$', 'normRB', 'normRBR', 'normRBS', 'normRP',
 'normSYM', 'normTO', 'normUH', 'normVB', 'normVBD', 'normVBG', 'normVBN',
 'normVBP', 'normVBZ', 'normWDT', 'normWP', 'normWP$', 'normWRB',
 'normXX', 'norm_SP', 'normDblSpecialApostrophe']])
#
dfOutTempNorm.loc[dfOutTempNorm['dollar']               == -1 , 'normDollar'] = -1
dfOutTempNorm.loc[dfOutTempNorm['dblApostrophe']        == -1 , 'normDblApostrophe'] = -1
dfOutTempNorm.loc[dfOutTempNorm['comma']                == -1 , 'normComma'] = -1
dfOutTempNorm.loc[dfOutTempNorm['-LRB-']                == -1 , 'norm-LRB-'] = -1
dfOutTempNorm.loc[dfOutTempNorm['-RRB-']                == -1 , 'norm-RRB-'] = -1
dfOutTempNorm.loc[dfOutTempNorm['dot']                  == -1 , 'normDot'] = -1
dfOutTempNorm.loc[dfOutTempNorm['colon']                == -1 , 'normColon'] = -1
dfOutTempNorm.loc[dfOutTempNorm['ADD']                  == -1 , 'normADD'] = -1
dfOutTempNorm.loc[dfOutTempNorm['AFX']                  == -1 , 'normAFX'] = -1
dfOutTempNorm.loc[dfOutTempNorm['CC']                   == -1 , 'normCC'] = -1
dfOutTempNorm.loc[dfOutTempNorm['CD']                   == -1 , 'normCD'] = -1
dfOutTempNorm.loc[dfOutTempNorm['DT']                   == -1 , 'normDT'] = -1
dfOutTempNorm.loc[dfOutTempNorm['EX']                   == -1 , 'normEX'] = -1
dfOutTempNorm.loc[dfOutTempNorm['FW']                   == -1 , 'normFW'] = -1
dfOutTempNorm.loc[dfOutTempNorm['HYPH']                 == -1 , 'normHYPH'] = -1
dfOutTempNorm.loc[dfOutTempNorm['IN']                   == -1 , 'normIN'] = -1
dfOutTempNorm.loc[dfOutTempNorm['JJ']                   == -1 , 'normJJ'] = -1
dfOutTempNorm.loc[dfOutTempNorm['JJR']                  == -1 , 'normJJR'] = -1
dfOutTempNorm.loc[dfOutTempNorm['JJS']                  == -1 , 'normJJS'] = -1
dfOutTempNorm.loc[dfOutTempNorm['LS']                   == -1 , 'normLS'] = -1
dfOutTempNorm.loc[dfOutTempNorm['MD']                   == -1 , 'normMD'] = -1
dfOutTempNorm.loc[dfOutTempNorm['NFP']                  == -1 , 'normNFP'] = -1
dfOutTempNorm.loc[dfOutTempNorm['NN']                   == -1 , 'normNN'] = -1
dfOutTempNorm.loc[dfOutTempNorm['NNPS']                 == -1 , 'normNNPS'] = -1
dfOutTempNorm.loc[dfOutTempNorm['NNS']                  == -1 , 'normNNS'] = -1
dfOutTempNorm.loc[dfOutTempNorm['PDT']                  == -1 , 'normPDT'] = -1
dfOutTempNorm.loc[dfOutTempNorm['POS']                  == -1 , 'normPOS'] = -1
dfOutTempNorm.loc[dfOutTempNorm['PRP']                  == -1 , 'normPRP'] = -1
dfOutTempNorm.loc[dfOutTempNorm['PRP$']                 == -1 , 'normPRP$'] = -1
dfOutTempNorm.loc[dfOutTempNorm['RB']                   == -1 , 'normRB'] = -1
dfOutTempNorm.loc[dfOutTempNorm['RBR']                  == -1 , 'normRBR'] = -1
dfOutTempNorm.loc[dfOutTempNorm['RBS']                  == -1 , 'normRBS'] = -1
dfOutTempNorm.loc[dfOutTempNorm['RP']                   == -1 , 'normRP'] = -1
dfOutTempNorm.loc[dfOutTempNorm['SYM']                  == -1 , 'normSYM'] = -1
dfOutTempNorm.loc[dfOutTempNorm['TO']                   == -1 , 'normTO'] = -1
dfOutTempNorm.loc[dfOutTempNorm['UH']                   == -1 , 'normUH'] = -1
dfOutTempNorm.loc[dfOutTempNorm['VB']                   == -1 , 'normVB'] = -1
dfOutTempNorm.loc[dfOutTempNorm['VBD']                  == -1 , 'normVBD'] = -1
dfOutTempNorm.loc[dfOutTempNorm['VBG']                  == -1 , 'normVBG'] = -1
dfOutTempNorm.loc[dfOutTempNorm['VBN']                  == -1 , 'normVBN'] = -1
dfOutTempNorm.loc[dfOutTempNorm['VBP']                  == -1 , 'normVBP'] = -1
dfOutTempNorm.loc[dfOutTempNorm['VBZ']                  == -1 , 'normVBZ'] = -1
dfOutTempNorm.loc[dfOutTempNorm['WDT']                  == -1 , 'normWDT'] = -1
dfOutTempNorm.loc[dfOutTempNorm['WP']                   == -1 , 'normWP'] = -1
dfOutTempNorm.loc[dfOutTempNorm['WP$']                  == -1 , 'normWP$'] = -1
dfOutTempNorm.loc[dfOutTempNorm['WRB']                  == -1 , 'normWRB'] = -1
dfOutTempNorm.loc[dfOutTempNorm['XX']                   == -1 , 'normXX'] = -1
dfOutTempNorm.loc[dfOutTempNorm['_SP']                  == -1 , 'norm_SP'] = -1
dfOutTempNorm.loc[dfOutTempNorm['dblSpecialApostrophe'] == -1 , 'normDblSpecialApostrophe'] = -1
#
dfOut['normDollar']               = dfOutTempNorm['normDollar']
dfOut['normDblApostrophe']        = dfOutTempNorm['normDblApostrophe']
dfOut['normComma']                = dfOutTempNorm['normComma']
dfOut['norm-LRB-']                = dfOutTempNorm['norm-LRB-']
dfOut['norm-RRB-']                = dfOutTempNorm['norm-RRB-']
dfOut['normDot']                  = dfOutTempNorm['normDot']
dfOut['normColon']                = dfOutTempNorm['normColon']
dfOut['normADD']                  = dfOutTempNorm['normADD']
dfOut['normAFX']                  = dfOutTempNorm['normAFX']
dfOut['normCC']                   = dfOutTempNorm['normCC']
dfOut['normCD']                   = dfOutTempNorm['normCD']
dfOut['normDT']                   = dfOutTempNorm['normDT']
dfOut['normEX']                   = dfOutTempNorm['normEX']
dfOut['normFW']                   = dfOutTempNorm['normFW']
dfOut['normHYPH']                 = dfOutTempNorm['normHYPH']
dfOut['normIN']                   = dfOutTempNorm['normIN']
dfOut['normJJ']                   = dfOutTempNorm['normJJ']
dfOut['normJJR']                  = dfOutTempNorm['normJJR']
dfOut['normJJS']                  = dfOutTempNorm['normJJS']
dfOut['normLS']                   = dfOutTempNorm['normLS']
dfOut['normMD']                   = dfOutTempNorm['normMD']
dfOut['normNFP']                  = dfOutTempNorm['normNFP']
dfOut['normNN']                   = dfOutTempNorm['normNN']
dfOut['normNNP']                  = dfOutTempNorm['normNNP']
dfOut['normNNPS']                 = dfOutTempNorm['normNNPS']
dfOut['normNNS']                  = dfOutTempNorm['normNNS']
dfOut['normPDT']                  = dfOutTempNorm['normPDT']
dfOut['normPOS']                  = dfOutTempNorm['normPOS']
dfOut['normPRP']                  = dfOutTempNorm['normPRP']
dfOut['normPRP$']                 = dfOutTempNorm['normPRP$']
dfOut['normRB']                   = dfOutTempNorm['normRB']
dfOut['normRBR']                  = dfOutTempNorm['normRBR']
dfOut['normRBS']                  = dfOutTempNorm['normRBS']
dfOut['normRP']                   = dfOutTempNorm['normRP']
dfOut['normSYM']                  = dfOutTempNorm['normSYM']
dfOut['normTO']                   = dfOutTempNorm['normTO']
dfOut['normUH']                   = dfOutTempNorm['normUH']
dfOut['normVB']                   = dfOutTempNorm['normVB']
dfOut['normVBD']                  = dfOutTempNorm['normVBD']
dfOut['normVBG']                  = dfOutTempNorm['normVBG']
dfOut['normVBN']                  = dfOutTempNorm['normVBN']
dfOut['normVBP']                  = dfOutTempNorm['normVBP']
dfOut['normVBZ']                  = dfOutTempNorm['normVBZ']
dfOut['normWDT']                  = dfOutTempNorm['normWDT']
dfOut['normWP']                   = dfOutTempNorm['normWP']
dfOut['normWP$']                  = dfOutTempNorm['normWP$']
dfOut['normWRB']                  = dfOutTempNorm['normWRB']
dfOut['normXX']                   = dfOutTempNorm['normXX']
dfOut['norm_SP']                  = dfOutTempNorm['norm_SP']
dfOut['normDblSpecialApostrophe'] = dfOutTempNorm['normDblSpecialApostrophe']
#

## writing the morphological features file

In [22]:
outFile_WITH_norm = '/home/rohit/SRH/CaseStudy1/dataDemo/dataFiles/temp_M_features.csv'
dfOut.to_csv(outFile_WITH_norm, index=False)
#
del dfOut
del dfOutTempNorm
#
### finished

# R features -- extract READABILITY features (using Textstat) -- using the title+content

In [23]:
# Use this script to process the datasets for READABILITY FEATURES EXTRACTION.
# Run the rogram with EXACTLY 5 argruments as shown below:
#       python <script> <input-file-name> <output-CSV-file> <#skipInitialRows> <#rowsToProcess> <#rowsAfterWhichToShowPrintMessageTracker>
#
#       #skipInitialRows
#               is an integer. Is the CSV data row number (0 being for header) from which the data
#               should be loaded. Here 0 is the header so to skip first 5 data rows: skipInitialRows = 5.
#       #rowsToProcess
#               is an integer. If = -1 then will read in all the rows after skipping as per above parameter.
#               Otherwise, will read in the number specified.
#
# The packages expected by the script are:
# import pandas as pd
# import numpy as np
# import spacy
# import nltk
# from nltk.tokenize.toktok import ToktokTokenizer
# import re
# from bs4 import BeautifulSoup
# import unicodedata
# import en_core_web_lg
# import en_core_web_sm
# from contractions import contractions_dict
# from textstat.textstat import textstatistics, easy_word_set, legacy_round
# from nltk.corpus import stopwords
# from nltk.tokenize import sent_tokenize, word_tokenize
# from sklearn.preprocessing import MinMaxScaler
# import csv
# import sys
# from datetime import datetime
# import logging

## python script will run and create the readability features file

### Note that the print statements about number columns in the dataframe.shape shows a value greater
###     by one (4 instead of 3, 23 instead of 22). This is fine. The script was written expecting the
###     input data to file to hold only the columns: domain, url, title and content.
###     But here was are running with an additional column for urlType and that is fine.

In [24]:
!python /home/rohit/SRH/CaseStudy1/dataDemo/code/scriptCreateReadabilityFeatures1.py /home/rohit/SRH/CaseStudy1/dataDemo/dataFiles/demorun_input_data.csv /home/rohit/SRH/CaseStudy1/dataDemo/dataFiles/temp_R_features.csv 0 15 1

All command line arguments are valid....starting main processing

Start time: Tue Oct 22 10:36:32 2019


Processing with command line arguments as:
1) dataIpFile       = /home/rohit/SRH/CaseStudy1/dataDemo/dataFiles/demorun_input_data.csv
2) csvOpFile        = /home/rohit/SRH/CaseStudy1/dataDemo/dataFiles/temp_R_features.csv
3) skipInitialRows  = 0
4) rowsToProcess    = 15
5) printMessageFreq = 1


Input data read into dataframe.


Normalized the full_text into clean_full_text

The shape should be = number of urls, 3 columns
data1.shape = (1, 4)
The shape should be = number of urls, 22 columns
data1.shape = (1, 23)


Starting the processing for readability features at:
Tue Oct 22 10:36:38 2019



Processed url = http://beforeitsnews.com/blogging-citizen-journalism/2018/01/attention-forward-to-president-trump-bombshell-breadcrumb-from-judge-anna-von-reitz-urgent-public-message-2585490.html
at row number = 0
The shape should be = number of urls, 21 columns, domain url, clean_text and 18 

# L features -- extract PSYCHOLOGICAL features (using LIWC)   -- using the title+content

In [ ]:
# This is a paid tool and there is no script for it. You pass it a CSV file with the textual data to be
#       analyzed and it returns 50 features.
# We are concatenating the Title+Content into so called "Full_Text" and using the tool to extract the
#       features for each URL.
# Thus, after the Psychological features are extracted using LIWC, it should be saved as a file:
#       temp_L_features.csv.
#
# The columns of the file should be as follows:
#
## So we will use the input file which has the domain, url, title, content.
#       Load it into dataframe, combine the title+content as "full_text".
#       Perform some basic cleaning on the "full_text" to create a new column "clean_full_text".
#       Then this columns data is extracted as a csv file containing only this "clean_full_text".
#       LIWC tool returns a csv file with columns as "clean_full_text" unchanged, followed by
#       93 features per row.
#
# This output file is the "temp_L_features.csv" which is used in the workflow.
#

# Combine the invidual features files for L, M, R and T into one file.

## Merging all the individual feature files into one

In [28]:
folderPath = '/home/rohit/SRH/CaseStudy1/dataDemo/dataFiles/'
skipInitialRows = 0

In [29]:
# merge all the features in order of M , L , T , R
inFile = folderPath + 'temp_M_features.csv'
dfTempM = pd.read_csv(inFile, skiprows = range(1, skipInitialRows + 1), sep = ',', low_memory=False)
inFile = folderPath + 'temp_L_features.csv'
dfTempL = pd.read_csv(inFile, skiprows = range(1, skipInitialRows + 1), sep = ',', low_memory=False)
inFile = folderPath + 'temp_T_features.csv'
dfTempT = pd.read_csv(inFile, skiprows = range(1, skipInitialRows + 1), sep = ',', low_memory=False)
inFile = folderPath + 'temp_R_features.csv'
dfTempR = pd.read_csv(inFile, skiprows = range(1, skipInitialRows + 1), sep = ',', low_memory=False)

In [30]:
######## temp code  ###   temp code   ###
#
# see the columns for each of the files
#
outColStr = ""
for col in list(dfTempM) :
    outColStr += col + ' , '
print(f"M features = \n{outColStr}\n")
#
outColStr = ""
for col in list(dfTempL) :
    outColStr += col + ' , '
print(f"L features = \n{outColStr}\n")
#
outColStr = ""
for col in list(dfTempT) :
    outColStr += col + ' , '
print(f"T features = \n{outColStr}\n")
#
outColStr = ""
for col in list(dfTempR) :
    outColStr += col + ' , '
print(f"R features = \n{outColStr}\n")
#
######## temp code  ###   temp code   ###

M features = 
domain , url , urlType , dollar , dblApostrophe , comma , -LRB- , -RRB- , dot , colon , ADD , AFX , CC , CD , DT , EX , FW , HYPH , IN , JJ , JJR , JJS , LS , MD , NFP , NN , NNP , NNPS , NNS , PDT , POS , PRP , PRP$ , RB , RBR , RBS , RP , SYM , TO , UH , VB , VBD , VBG , VBN , VBP , VBZ , WDT , WP , WP$ , WRB , XX , _SP , dblSpecialApostrophe , normDollar , normDblApostrophe , normComma , norm-LRB- , norm-RRB- , normDot , normColon , normADD , normAFX , normCC , normCD , normDT , normEX , normFW , normHYPH , normIN , normJJ , normJJR , normJJS , normLS , normMD , normNFP , normNN , normNNP , normNNPS , normNNS , normPDT , normPOS , normPRP , normPRP$ , normRB , normRBR , normRBS , normRP , normSYM , normTO , normUH , normVB , normVBD , normVBG , normVBN , normVBP , normVBZ , normWDT , normWP , normWP$ , normWRB , normXX , norm_SP , normDblSpecialApostrophe , 

L features = 
domain , url , WC , Analytic , Clout , Authentic , Tone , WPS , Sixltr , Dic , function , pronoun

In [31]:
# At this stage domain and urlType exist in multiple files, but we want to retain only the M file ones.
# delete the domain repetition and only retain it in the T file
del dfTempL['domain']
del dfTempR['domain']
del dfTempR['urlType']
# T file does not have domain.
#
dfTempOut = pd.merge(dfTempM, dfTempL, on=['url'])
dfTempOut1 = pd.merge(dfTempOut, dfTempT, on=['url'])
del dfTempOut
df_ALL_Out_WITH_norm = pd.merge(dfTempOut1, dfTempR, on=['url'])
del dfTempOut1
#
######## temp code  ###   temp code   ###
#
# see the columns for each of the files
#
outColStr = ""
for col in list(df_ALL_Out_WITH_norm) :
    outColStr += col + ' , '
print(f"df_ALL_Out_WITH_norm features = \n{outColStr}\n")
#
######## temp code  ###   temp code   ###

df_ALL_Out_WITH_norm features = 
domain , url , urlType , dollar , dblApostrophe , comma , -LRB- , -RRB- , dot , colon , ADD , AFX , CC , CD , DT , EX , FW , HYPH , IN , JJ , JJR , JJS , LS , MD , NFP , NN , NNP , NNPS , NNS , PDT , POS , PRP , PRP$ , RB , RBR , RBS , RP , SYM , TO , UH , VB , VBD , VBG , VBN , VBP , VBZ , WDT , WP , WP$ , WRB , XX , _SP , dblSpecialApostrophe , normDollar , normDblApostrophe , normComma , norm-LRB- , norm-RRB- , normDot , normColon , normADD , normAFX , normCC , normCD , normDT , normEX , normFW , normHYPH , normIN , normJJ , normJJR , normJJS , normLS , normMD , normNFP , normNN , normNNP , normNNPS , normNNS , normPDT , normPOS , normPRP , normPRP$ , normRB , normRBR , normRBS , normRP , normSYM , normTO , normUH , normVB , normVBD , normVBG , normVBN , normVBP , normVBZ , normWDT , normWP , normWP$ , normWRB , normXX , norm_SP , normDblSpecialApostrophe , WC , Analytic , Clout , Authentic , Tone , WPS , Sixltr , Dic , function , pronoun , ppron , i

In [32]:
colsList_ALL_WITH_norm = []
# expecting the order of features as Morphological , LIWC (Psychological) , Twitter , Readability
preTagList = ['' , 'M_' , 'L_' , 'T_' , 'R_']
idxPreTagList = 0
for col in list(df_ALL_Out_WITH_norm) :
    if col == 'dollar':                            ## first column of M Morphological
        idxPreTagList = 1
    elif col == 'WC':                              ## first column of L LIWC
        idxPreTagList = 2
    elif col == 'totalTweets':                     ## first column of T Twitter
        idxPreTagList = 3
    elif col == 'full_text_Flesch_reading_ease':   ## first column of R Readability
        idxPreTagList = 4
    newCol = preTagList[idxPreTagList] + col
    colsList_ALL_WITH_norm.append(newCol)
#
# Update the df_All_Out columns
# Thus the All features WITH norm dataframe is ready
#
df_ALL_Out_WITH_norm.columns = colsList_ALL_WITH_norm
#
######## temp code  ###   temp code   ###
#
# see the updated column names for all the features
#
outColStr = ""
for col in list(df_ALL_Out_WITH_norm) :
    outColStr += col + ' , '
print(f"df_ALL_Out_WITH_norm features after updation =\n{outColStr}\n")
#
######## temp code  ###   temp code   ###
#
#
########
#
# identify ALL features which are regular by dropping normalized features and write to CSV file
colsList_ALL_NO_norm = []
for col in list(df_ALL_Out_WITH_norm) :
    if col in ['url' , 'domain' , 'urlType']: # these three must be retained
        colsList_ALL_NO_norm.append(col)
    elif "norm" not in col :
        colsList_ALL_NO_norm.append(col)
#
df_ALL_Out_NO_norm = df_ALL_Out_WITH_norm[colsList_ALL_NO_norm]
#
######## temp code  ###   temp code   ###
#
# check the columns
#
outColStr = ""
for col in list(colsList_ALL_NO_norm) :
    outColStr += col + ' , '
print(f"df_ALL_Out_NO_norm features =\n{outColStr}\n")
#
######## temp code  ###   temp code   ###
#

df_ALL_Out_WITH_norm features after updation =
domain , url , urlType , M_dollar , M_dblApostrophe , M_comma , M_-LRB- , M_-RRB- , M_dot , M_colon , M_ADD , M_AFX , M_CC , M_CD , M_DT , M_EX , M_FW , M_HYPH , M_IN , M_JJ , M_JJR , M_JJS , M_LS , M_MD , M_NFP , M_NN , M_NNP , M_NNPS , M_NNS , M_PDT , M_POS , M_PRP , M_PRP$ , M_RB , M_RBR , M_RBS , M_RP , M_SYM , M_TO , M_UH , M_VB , M_VBD , M_VBG , M_VBN , M_VBP , M_VBZ , M_WDT , M_WP , M_WP$ , M_WRB , M_XX , M__SP , M_dblSpecialApostrophe , M_normDollar , M_normDblApostrophe , M_normComma , M_norm-LRB- , M_norm-RRB- , M_normDot , M_normColon , M_normADD , M_normAFX , M_normCC , M_normCD , M_normDT , M_normEX , M_normFW , M_normHYPH , M_normIN , M_normJJ , M_normJJR , M_normJJS , M_normLS , M_normMD , M_normNFP , M_normNN , M_normNNP , M_normNNPS , M_normNNS , M_normPDT , M_normPOS , M_normPRP , M_normPRP$ , M_normRB , M_normRBR , M_normRBS , M_normRP , M_normSYM , M_normTO , M_normUH , M_normVB , M_normVBD , M_normVBG , M_normVBN , M_n

# create all the data files to be used for training or testing new data

In [33]:
folderPath = '/home/rohit/SRH/CaseStudy1/dataDemo/dataFiles/'

### write to CSV: All features WITH norm    and   other file for All features WITHOUT norm

In [34]:
df_ALL_Out_WITH_norm.to_csv(folderPath + 'All_Features_WITH_norm.csv' , index = True)
df_ALL_Out_NO_norm.to_csv(folderPath + 'All_Features_NO_norm.csv' , index = True)

In [35]:
del df_ALL_Out_WITH_norm , df_ALL_Out_NO_norm, dfTempM , dfTempL , dfTempT , dfTempR
del colsList_ALL_WITH_norm, colsList_ALL_NO_norm, outColStr

# Read in the All_Features file into a dataframe and process from there to create
#      the individual group feature files

In [36]:
df_ALL_Out_WITH_norm = pd.read_csv(folderPath + 'All_Features_WITH_norm.csv' , index_col = 0, sep = ',', low_memory=False)

In [37]:
df_ALL_Out_WITH_norm.shape

(1, 378)

In [38]:
outColStr = ""
for col in list(df_ALL_Out_WITH_norm) :
    outColStr += col + ' , '
print(f"df_ALL_Out_WITH_norm features after updation =\n{outColStr}\n")

df_ALL_Out_WITH_norm features after updation =
domain , url , urlType , M_dollar , M_dblApostrophe , M_comma , M_-LRB- , M_-RRB- , M_dot , M_colon , M_ADD , M_AFX , M_CC , M_CD , M_DT , M_EX , M_FW , M_HYPH , M_IN , M_JJ , M_JJR , M_JJS , M_LS , M_MD , M_NFP , M_NN , M_NNP , M_NNPS , M_NNS , M_PDT , M_POS , M_PRP , M_PRP$ , M_RB , M_RBR , M_RBS , M_RP , M_SYM , M_TO , M_UH , M_VB , M_VBD , M_VBG , M_VBN , M_VBP , M_VBZ , M_WDT , M_WP , M_WP$ , M_WRB , M_XX , M__SP , M_dblSpecialApostrophe , M_normDollar , M_normDblApostrophe , M_normComma , M_norm-LRB- , M_norm-RRB- , M_normDot , M_normColon , M_normADD , M_normAFX , M_normCC , M_normCD , M_normDT , M_normEX , M_normFW , M_normHYPH , M_normIN , M_normJJ , M_normJJR , M_normJJS , M_normLS , M_normMD , M_normNFP , M_normNN , M_normNNP , M_normNNPS , M_normNNS , M_normPDT , M_normPOS , M_normPRP , M_normPRP$ , M_normRB , M_normRBR , M_normRBS , M_normRP , M_normSYM , M_normTO , M_normUH , M_normVB , M_normVBD , M_normVBG , M_normVBN , M_n

# identify L features and write to CSV file
### this is done for two files:
###        one WITH normalized values included
###        other is for NO normalized values

In [39]:
# identify all L features and write to CSV file
colsList_L_WITH_norm = []
for col in list(df_ALL_Out_WITH_norm) :
    if col in ['url' , 'domain' , 'urlType']: # these three must be retained
        colsList_L_WITH_norm.append(col)
    elif col.startswith('L_'):
        colsList_L_WITH_norm.append(col)
#
df_L_Out_WITH_norm = df_ALL_Out_WITH_norm[colsList_L_WITH_norm]
#
######## temp code  ###   temp code   ###
#
# check the columns
#
outColStr = ""
for col in list(df_L_Out_WITH_norm) :
    outColStr += col + ' , '
print(f"df_L_Out_WITH_norm features =\n{outColStr}\n")
#
######## temp code  ###   temp code   ###
#
#
########
#
# identify L features which are regular by dropping normalized features and write to CSV file
colsList_L_NO_norm = []
for col in list(df_ALL_Out_WITH_norm) :
    if col in ['url' , 'domain' , 'urlType']: # these three must be retained
        colsList_L_NO_norm.append(col)
    elif col.startswith('L_'):
        if "norm" not in col :
            colsList_L_NO_norm.append(col)
#
df_L_Out_NO_norm = df_ALL_Out_WITH_norm[colsList_L_NO_norm]
#
######## temp code  ###   temp code   ###
#
# check the columns
#
outColStr = ""
for col in list(df_L_Out_NO_norm) :
    outColStr += col + ' , '
print(f"df_L_Out_NO_norm features =\n{outColStr}\n")
#
######## temp code  ###   temp code   ###
#

df_L_Out_WITH_norm features =
domain , url , urlType , L_WC , L_Analytic , L_Clout , L_Authentic , L_Tone , L_WPS , L_Sixltr , L_Dic , L_function , L_pronoun , L_ppron , L_i , L_we , L_you , L_shehe , L_they , L_ipron , L_article , L_prep , L_auxverb , L_adverb , L_conj , L_negate , L_verb , L_adj , L_compare , L_interrog , L_number , L_quant , L_affect , L_posemo , L_negemo , L_anx , L_anger , L_sad , L_social , L_family , L_friend , L_female , L_male , L_cogproc , L_insight , L_cause , L_discrep , L_tentat , L_certain , L_differ , L_percept , L_see , L_hear , L_feel , L_bio , L_body , L_health , L_sexual , L_ingest , L_drives , L_affiliation , L_achieve , L_power , L_reward , L_risk , L_focuspast , L_focuspresent , L_focusfuture , L_relativ , L_motion , L_space , L_time , L_work , L_leisure , L_home , L_money , L_relig , L_death , L_informal , L_swear , L_netspeak , L_assent , L_nonflu , L_filler , L_AllPunc , L_Period , L_Comma , L_Colon , L_SemiC , L_QMark , L_Exclam , L_Dash , L_Q

### write to CSV: L features WITH norm    and   other file for L features WITHOUT norm

In [40]:
df_L_Out_WITH_norm.to_csv(folderPath + 'L_Features_WITH_norm.csv' , index = True)
del df_L_Out_WITH_norm
df_L_Out_NO_norm.to_csv(folderPath + 'L_Features_NO_norm.csv' , index = True)
del df_L_Out_NO_norm

# identify M features and write to CSV file
### this is done for two files:
###        one WITH normalized values included
###        other is for NO normalized values

In [41]:
# identify all M features and write to CSV file
colsList_M_WITH_norm = []
for col in list(df_ALL_Out_WITH_norm) :
    if col in ['url' , 'domain' , 'urlType']: # these three must be retained
        colsList_M_WITH_norm.append(col)
    elif col.startswith('M_'):
        colsList_M_WITH_norm.append(col)
#
df_M_Out_WITH_norm = df_ALL_Out_WITH_norm[colsList_M_WITH_norm]
#
######## temp code  ###   temp code   ###
#
# check the columns
#
outColStr = ""
for col in list(df_M_Out_WITH_norm) :
    outColStr += col + ' , '
print(f"df_M_Out_WITH_norm features =\n{outColStr}\n")
#
######## temp code  ###   temp code   ###
#
#
########
#
# identify M features which are regular by dropping normalized features and write to CSV file
colsList_M_NO_norm = []
for col in list(df_ALL_Out_WITH_norm) :
    if col in ['url' , 'domain' , 'urlType']: # these three must be retained
        colsList_M_NO_norm.append(col)
    elif col.startswith('M_'):
        if "norm" not in col :
            colsList_M_NO_norm.append(col)
#
df_M_Out_NO_norm = df_ALL_Out_WITH_norm[colsList_M_NO_norm]
#
######## temp code  ###   temp code   ###
#
# check the columns
#
outColStr = ""
for col in list(df_M_Out_NO_norm) :
    outColStr += col + ' , '
print(f"df_M_Out_NO_norm features =\n{outColStr}\n")
#
######## temp code  ###   temp code   ###
#

df_M_Out_WITH_norm features =
domain , url , urlType , M_dollar , M_dblApostrophe , M_comma , M_-LRB- , M_-RRB- , M_dot , M_colon , M_ADD , M_AFX , M_CC , M_CD , M_DT , M_EX , M_FW , M_HYPH , M_IN , M_JJ , M_JJR , M_JJS , M_LS , M_MD , M_NFP , M_NN , M_NNP , M_NNPS , M_NNS , M_PDT , M_POS , M_PRP , M_PRP$ , M_RB , M_RBR , M_RBS , M_RP , M_SYM , M_TO , M_UH , M_VB , M_VBD , M_VBG , M_VBN , M_VBP , M_VBZ , M_WDT , M_WP , M_WP$ , M_WRB , M_XX , M__SP , M_dblSpecialApostrophe , M_normDollar , M_normDblApostrophe , M_normComma , M_norm-LRB- , M_norm-RRB- , M_normDot , M_normColon , M_normADD , M_normAFX , M_normCC , M_normCD , M_normDT , M_normEX , M_normFW , M_normHYPH , M_normIN , M_normJJ , M_normJJR , M_normJJS , M_normLS , M_normMD , M_normNFP , M_normNN , M_normNNP , M_normNNPS , M_normNNS , M_normPDT , M_normPOS , M_normPRP , M_normPRP$ , M_normRB , M_normRBR , M_normRBS , M_normRP , M_normSYM , M_normTO , M_normUH , M_normVB , M_normVBD , M_normVBG , M_normVBN , M_normVBP , M_normVB

### write to CSV: M features WITH norm    and   other file for M features WITHOUT norm

In [42]:
df_M_Out_WITH_norm.to_csv(folderPath + 'M_Features_WITH_norm.csv' , index = True)
del df_M_Out_WITH_norm
df_M_Out_NO_norm.to_csv(folderPath + 'M_Features_NO_norm.csv' , index = True)
del df_M_Out_NO_norm

# identify R features and write to CSV file
### this is done for two files:
###        one WITH normalized values included
###        other is for NO normalized values

In [43]:
# identify all R features and write to CSV file
colsList_R_WITH_norm = []
for col in list(df_ALL_Out_WITH_norm) :
    if col in ['url' , 'domain' , 'urlType']: # these three must be retained
        colsList_R_WITH_norm.append(col)
    elif col.startswith('R_'):
        colsList_R_WITH_norm.append(col)
#
df_R_Out_WITH_norm = df_ALL_Out_WITH_norm[colsList_R_WITH_norm]
#
######## temp code  ###   temp code   ###
#
# check the columns
#
outColStr = ""
for col in list(df_R_Out_WITH_norm) :
    outColStr += col + ' , '
print(f"df_R_Out_WITH_norm features =\n{outColStr}\n")
#
######## temp code  ###   temp code   ###
#
#
########
#
# identify R features which are regular by dropping normalized features and write to CSV file
colsList_R_NO_norm = []
for col in list(df_ALL_Out_WITH_norm) :
    if col in ['url' , 'domain' , 'urlType']: # these three must be retained
        colsList_R_NO_norm.append(col)
    elif col.startswith('R_'):
        if "norm" not in col :
            colsList_R_NO_norm.append(col)
#
df_R_Out_NO_norm = df_ALL_Out_WITH_norm[colsList_R_NO_norm]
#
######## temp code  ###   temp code   ###
#
# check the columns
#
outColStr = ""
for col in list(df_R_Out_NO_norm) :
    outColStr += col + ' , '
print(f"df_R_Out_NO_norm features =\n{outColStr}\n")
#
######## temp code  ###   temp code   ###
#

df_R_Out_WITH_norm features =
domain , url , urlType , R_full_text_Flesch_reading_ease , R_full_text_Flesch_kincaid_grade , R_full_text_smog , R_full_text_gunning_fog , R_full_text_words_per_sentence , R_full_text_capitalized_words , R_full_text_lexicon , R_full_text_urls_counts , R_full_text_long_words , R_full_text_syllables , R_full_text_stop_words , R_full_text_sentences , R_full_text_linsear_write , R_full_text_automated_readability , R_full_text_characters_total , R_full_text_coleman_liax , R_full_text_difficult_words , R_full_text_words_total , R_norm_full_text_Flesch_reading_ease , R_norm_full_text_Flesch_kincaid_grade , R_norm_full_text_smog , R_norm_full_text_gunning_fog , R_norm_full_text_words_per_sentence , R_norm_full_text_capitalized_words , R_norm_full_text_lexicon , R_norm_full_text_urls_counts , R_norm_full_text_long_words , R_norm_full_text_syllables , R_norm_full_text_stop_words , R_norm_full_text_sentences , R_norm_full_text_linsear_write , R_norm_full_text_automat

In [44]:
df_R_Out_WITH_norm.to_csv(folderPath + 'R_Features_WITH_norm.csv' , index = True)
del df_R_Out_WITH_norm
df_R_Out_NO_norm.to_csv(folderPath + 'R_Features_NO_norm.csv' , index = True)
del df_R_Out_NO_norm

# identify T features and write to CSV file
### this is done for two files:
###        one WITH normalized values included
###        other is for NO normalized values

In [45]:
# identify all T features and write to CSV file
colsList_T_WITH_norm = []
for col in list(df_ALL_Out_WITH_norm) :
    if col in ['url' , 'domain' , 'urlType']: # these three must be retained
        colsList_T_WITH_norm.append(col)
    elif col.startswith('T_'):
        colsList_T_WITH_norm.append(col)
#
df_T_Out_WITH_norm = df_ALL_Out_WITH_norm[colsList_T_WITH_norm]
#
######## temp code  ###   temp code   ###
#
# check the columns
#
outColStr = ""
for col in list(df_T_Out_WITH_norm) :
    outColStr += col + ' , '
print(f"df_T_Out_WITH_norm features =\n{outColStr}\n")
#
######## temp code  ###   temp code   ###
#
#
########
#
# identify T features which are regular by dropping normalized features and write to CSV file
colsList_T_NO_norm = []
for col in list(df_ALL_Out_WITH_norm) :
    if col in ['url' , 'domain' , 'urlType']: # these three must be retained
        colsList_T_NO_norm.append(col)
    elif col.startswith('T_'):
        if "norm" not in col :
            colsList_T_NO_norm.append(col)
#
df_T_Out_NO_norm = df_ALL_Out_WITH_norm[colsList_T_NO_norm]
#
######## temp code  ###   temp code   ###
#
# check the columns
#
outColStr = ""
for col in list(df_T_Out_NO_norm) :
    outColStr += col + ' , '
print(f"df_T_Out_NO_norm features =\n{outColStr}\n")
#
######## temp code  ###   temp code   ###
#

df_T_Out_WITH_norm features =
domain , url , urlType , T_totalTweets , T_totalRetweets , T_totalLikes , T_totalReplies , T_span , T_countWeekDay , T_countWeekEnd , T_countMonday , T_countTuesday , T_countWednesday , T_countThursday , T_countFriday , T_countSaturday , T_countSunday , T_avgTimeBetweenTweets , T_avgTimeOfNextTweet , T_timeAbsBin1count0to6 , T_timeAbsBin2count6to12 , T_timeAbsBin3count12to18 , T_timeAbsBin4count18to24 , T_timeAbsBin5count24plus , T_avgTweetsPerUniqUser , T_normTotalTweets , T_normTotalRetweets , T_normTotalLikes , T_normTotalReplies , T_normCountWeekDay , T_normCountWeekEnd , T_normCountMonday , T_normCountTuesday , T_normCountWednesday , T_normCountThursday , T_normCountFriday , T_normCountSaturday , T_normCountSunday , T_normAvgTweetsPerUniqUser , T_normTimeAbsBin1count0to6 , T_normTimeAbsBin2count6to12 , T_normTimeAbsBin3count12to18 , T_normTimeAbsBin4count18to24 , T_normTimeAbsBin5count24plus , T_normSpan , T_normAvgTimeBetweenTweets , T_normAvgTimeOfN

In [46]:
df_T_Out_WITH_norm.to_csv(folderPath + 'T_Features_WITH_norm.csv' , index = True)
del df_T_Out_WITH_norm
df_T_Out_NO_norm.to_csv(folderPath + 'T_Features_NO_norm.csv' , index = True)
del df_T_Out_NO_norm

# identify MLR features and write to CSV file
### this is done for two files:
###        one WITH normalized values included
###        other is for NO normalized values

In [47]:
# identify all MLR features and write to CSV file
colsList_MLR_WITH_norm = []
for col in list(df_ALL_Out_WITH_norm) :
    if col in ['url' , 'domain' , 'urlType']: # these three must be retained
        colsList_MLR_WITH_norm.append(col)
    elif col.startswith('M_') or col.startswith('L_') or col.startswith('R_'):
        colsList_MLR_WITH_norm.append(col)
#
df_MLR_Out_WITH_norm = df_ALL_Out_WITH_norm[colsList_MLR_WITH_norm]
#
######## temp code  ###   temp code   ###
#
# check the columns
#
outColStr = ""
for col in list(df_MLR_Out_WITH_norm) :
    outColStr += col + ' , '
print(f"df_MLR_Out_WITH_norm features =\n{outColStr}\n")
#
######## temp code  ###   temp code   ###
#
#
########
#
# identify MLR features which are regular by dropping normalized features and write to CSV file
colsList_MLR_NO_norm = []
for col in list(df_ALL_Out_WITH_norm) :
    if col in ['url' , 'domain' , 'urlType']: # these three must be retained
        colsList_MLR_NO_norm.append(col)
    elif col.startswith('M_') or col.startswith('L_') or col.startswith('R_') :
        if "norm" not in col :
            colsList_MLR_NO_norm.append(col)
#
df_MLR_Out_NO_norm = df_ALL_Out_WITH_norm[colsList_MLR_NO_norm]
#
######## temp code  ###   temp code   ###
#
# check the columns
#
outColStr = ""
for col in list(df_MLR_Out_NO_norm) :
    outColStr += col + ' , '
print(f"df_MLR_Out_NO_norm features =\n{outColStr}\n")
#
######## temp code  ###   temp code   ###
#

df_MLR_Out_WITH_norm features =
domain , url , urlType , M_dollar , M_dblApostrophe , M_comma , M_-LRB- , M_-RRB- , M_dot , M_colon , M_ADD , M_AFX , M_CC , M_CD , M_DT , M_EX , M_FW , M_HYPH , M_IN , M_JJ , M_JJR , M_JJS , M_LS , M_MD , M_NFP , M_NN , M_NNP , M_NNPS , M_NNS , M_PDT , M_POS , M_PRP , M_PRP$ , M_RB , M_RBR , M_RBS , M_RP , M_SYM , M_TO , M_UH , M_VB , M_VBD , M_VBG , M_VBN , M_VBP , M_VBZ , M_WDT , M_WP , M_WP$ , M_WRB , M_XX , M__SP , M_dblSpecialApostrophe , M_normDollar , M_normDblApostrophe , M_normComma , M_norm-LRB- , M_norm-RRB- , M_normDot , M_normColon , M_normADD , M_normAFX , M_normCC , M_normCD , M_normDT , M_normEX , M_normFW , M_normHYPH , M_normIN , M_normJJ , M_normJJR , M_normJJS , M_normLS , M_normMD , M_normNFP , M_normNN , M_normNNP , M_normNNPS , M_normNNS , M_normPDT , M_normPOS , M_normPRP , M_normPRP$ , M_normRB , M_normRBR , M_normRBS , M_normRP , M_normSYM , M_normTO , M_normUH , M_normVB , M_normVBD , M_normVBG , M_normVBN , M_normVBP , M_norm

In [48]:
df_MLR_Out_WITH_norm.to_csv(folderPath + 'MLR_Features_WITH_norm.csv' , index = True)
del df_MLR_Out_WITH_norm
df_MLR_Out_NO_norm.to_csv(folderPath + 'MLR_Features_NO_norm.csv' , index = True)
del df_MLR_Out_NO_norm

## Now all the output files with features are ready:
### For WITH normal : one each for All, M, L, T, R, MLR = 6 files
### For NO   normal : one each for All, M, L, T, R, MLR = 6 files
### Thus total 12 files
## ------------------------------------------------------------------------
# We will be using the NO normal versions for testing the accuracy of models

In [49]:
####################################################
####################################################
####################################################
####################################################
####################################################
####################################################

# This data can now be used either for training of classifiers or to use as new data to present to already trained models pickle file

In [ ]:
# https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/